In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary
import pickle
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import io
import os
import re
import numpy as np
import pickle
import pandas as pd

### Collect urls for pages with property listings

In [349]:
# Urls for Airbnb searches for Seattle

'''Urls 1-6 are from multiple searches for "Seattle" as location. One search yields ~300 properties. 
   Subsequent searches give duplicate urls. To inclrease the number of not duplicate urls used keyword 
   search for links 7-14. E.g. link#10 - propoerties that have 2 or more beadrooms.
   
'''
seattle1 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?adults=1&children=0&checkin=&checkout=&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&refinement_paths%5B%5D=%2Fhomes'
seattle2 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?adults=1&children=0&checkin=&checkout=&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&refinement_paths%5B%5D=%2Fhomes'
seattle3 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?adults=1&children=0&checkin=&checkout=&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&refinement_paths%5B%5D=%2Fhomes'
seattle4 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?adults=1&children=0&checkin=&checkout=&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A63ffe2ee-477d-4903-9f71-e40b3fad5481%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&refinement_paths%5B%5D=%2Fhomes'
seattle5 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?adults=1&children=0&checkin=&checkout=&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A63ffe2ee-477d-4903-9f71-e40b3fad5481%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&refinement_paths%5B%5D=%2Fhomes'
seattle6 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A63ffe2ee-477d-4903-9f71-e40b3fad5481%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&s_tag=tLFHWR38&search_type=pagination&screen_size=large&hide_dates_and_guests_filters=false&adults=1&last_search_session_id=d7f2e161-cce8-44a1-86d6-b59bce0f84ea'
# pets
seattle7 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A63ffe2ee-477d-4903-9f71-e40b3fad5481%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&adults=1&amenities%5B%5D=12'
# smoking
seattle8 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&amenities%5B%5D=11'
# 2 beds
seattle9 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&min_beds=2'
# 2 bedrooms
seattle10 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&min_beds=0&min_bedrooms=2'
# kitchen
seattle11 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&refinement_paths%5B%5D=%2Fhomes&adults=1&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&amenities%5B%5D=8&search_type=filter_change'
# 3 bedrooms
seattle12 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&adults=1&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&min_bedrooms=3'
#  3 beds
seattle13 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&adults=1&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&min_bedrooms=0&min_bathrooms=0&min_beds=3'
# property type - house
seattle14 = 'https://www.airbnb.com/s/Seattle--WA--United-States/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&source=mc_search_bar&click_referer=t%3ASEE_ALL%7Csid%3A1ce0219a-a643-4c55-a468-ec29a0069782%7Cst%3ALANDING_PAGE_MARQUEE&title_type=NONE&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=filter_change&screen_size=large&hide_dates_and_guests_filters=false&adults=1&min_beds=0&property_type_id%5B%5D=2'

location_urls_1 = [
    seattle1, seattle2, seattle3, seattle4, seattle5,
    seattle6, seattle7, seattle8, seattle9, seattle10
]

location_urls_2 = [
    seattle11, seattle12, seattle13, seattle14
]

In [357]:
# Collecting property listings.
dict_list_1 = collect_prop_urls(location_urls_1)
dict_list_2 = collect_prop_urls(location_urls_2)

NameError: name 'dict_list2' is not defined

In [365]:
# pickle page urls
with open('dict_list_1.pkl', 'wb') as f:
    pickle.dump(dict_list_1, f)
    
with open('dict_list_2.pkl', 'wb') as f:
    pickle.dump(dict_list_2, f)

In [366]:
# Number of urls collected for each link for dict_list_1
for dict in dict_list_1:
    print(len(dict))

262
247
255
257


In [ ]:
# Number of urls collected for each link for dict_list_2
for dict in dict_list_2:
    print(len(dict))

In [367]:
dict_list_all = dict_list_1 + dict_list_2

In [422]:
# Remove duplicate urls
dict_list_un_all = unique_prop_urls(dict_list_all)

In [423]:
# Number of urls collected for each link for dict_list_un_all
for dict in dict_list_un_all:
    print(len(dict))
    

278
122
22
43
29
9
216
157
134
182
40
184
92
128


In [375]:
# pickle page urls
with open('dict_lits_un_all.pkl', 'wb') as f:
    pickle.dump(dict_lits_un_all, f)

In [151]:
# Function

def collect_prop_urls(url_list):
    dict_list = []
    for url in url_list:
        page_urls = collect_page_urls(url)
        element_urls = collect_objct_urls(page_urls)
        prop_urls = select_prop_urls(element_urls)
        prop_urls_dict = make_dict_prop_id_url(prop_urls)
        
      # check for unique properties
#         prop_urls_dict_un = set(prop_urls_dict)
        dict_list.append(prop_urls_dict)
    return dict_list

In [323]:
# Function

def unique_prop_urls(url_dict_list):
    url_dict_list_un = []
    url_dict_list_un.append(url_dict_list[0])
    preceding_dicts = url_dict_list[0].copy() 
    
    for dict in url_dict_list[1:]:
        url_dict_un = check_dict(dict, preceding_dicts)
        
        preceding_dicts.update(url_dict_un)
        
        url_dict_list_un.append(url_dict_un)
        
    return url_dict_list_un

In [161]:
# Function

def check_dict(dict1, dict2):
    dict_w_unique_properties = {}
    for key, val in dict1.items():
        if key not in dict2:
            dict_w_unique_properties[key] = val
    return dict_w_unique_properties

In [132]:
# Function: Collect a list of page urls 

def collect_page_urls(start_url):
    driver = webdriver.Chrome()
    driver.get(start_url)

# add starting page 1 url
    page_urls_list = []
    page_urls_list.append(driver.current_url)
    time.sleep(7)

# move to page 2
    next_page = driver.find_elements_by_class_name("_1m76pmy")[0]
    next_page.click()

    time.sleep(3)

# collect urls and move to subsequent pages
    while True:
        page_url = driver.current_url
        page_urls_list.append(page_url)
        try:
            next_page = driver.find_elements_by_class_name("_1m76pmy")[1]
        except IndexError:
            break
        next_page.click()
        time.sleep(7)
    
    return page_urls_list

### Collect urls for property listings

In [133]:
# Function: Collect urls from webpages for each location

def collect_objct_urls(page_urls_list):      
    link_list = []
    for url in page_urls_list:     
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(5)
        link_elements = driver.find_elements_by_tag_name('a')
        for element in link_elements:
            try:                                                 # handling stale element error
                link = element.get_attribute('href')
                link_list.append(link)
            except StaleElementReferenceException:
                pass
        driver.close()
    return link_list

In [436]:
# Make combined dict with all urls
dict_combined = {k:v for d in dict_list_un_all for k, v in d.items() }
# for k, v in [x.items() for x in lst]:
len(dict_combined)

1636

In [134]:
# Function: Select property urls

def select_prop_urls(url_list):
            
# make sure all links are str type
    link_list_str = []
    for element in url_list:
        if type(element)==str:
            link_list_str.append(element)
        
# sort for property urls 
    property_urls = []
    text = 'name=1000'
    for url in link_list_str:
        if url.find(text) >=0:
            property_urls.append(url)                     
        
# remove duplicates       
    property_urls = set(property_urls)
    return property_urls

In [121]:
seattle_prop_urls3 = select_prop_urls(seattle_obsr_urls3)

In [18]:
# Function: extract property ID from url

def url_to_id(url):
    to_num = lambda num: int(''.join(i for i in num)) 
    prop_id = to_num(re.findall(r'\d', url.split('?')[0]))
    return prop_id

In [135]:
# Function: Make dict with urls and property IDs

def make_dict_prop_id_url(url_list):
    dict_id_url = {}
    for url in url_list:
        dict_id_url[url_to_id(url)] = url
    return dict_id_url

In [123]:
# Make dict with url IDs and urls
# seattle_prop_urls_dict = make_dict_prop_id_url(seattle_prop_urls)
# shoreline_prop_urls_dict = make_dict_prop_id_url(shoreline_prop_urls)
# bellevue_prop_urls_dict = make_dict_prop_id_url(bellevue_prop_urls)
# redmond_prop_urls_dict = make_dict_prop_id_url(redmond_prop_urls)
# seattle_prop_urls_dict2 = make_dict_prop_id_url(seattle_prop_urls2)
seattle_prop_urls_dict3 = make_dict_prop_id_url(seattle_prop_urls3)


In [136]:
# Function: make a dict with not repeating entries as compared to another dict

def check_dict(dict1, dict2):
    dict_w_unique_properties = {}
    for key, val in dict1.items():
        if key not in dict2:
            dict_w_unique_properties[key] = val
    return dict_w_unique_properties

In [ ]:
dict_combined

In [533]:
# pickle prop_urls
with open('dict_combined.pkl', 'wb') as f:
    pickle.dump(dict_combined, f)

## Extract raw features

In [4]:
cols = ['property_id', 'price', 'guests', 'bedrooms', 'beds', 
        'baths', 'latitude', 'longitude', 'review_rate', 'review_num', 
        'property_type', 'link']
df_raw = pd.DataFrame(columns = cols)

In [558]:
# use selenium to extract raw features from a single web page
def get_raw_feature_row(prop_id, url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(7) 
    
# set raw features to nan
    property_id, price, guests, bedrooms, beds, baths, property_type, review_rate, review_num, link, latitude, longitude = [np.nan]*12
    
    property_id = prop_id
    link = driver.current_url
    
# get raw features
    try:
        price = int(driver.find_element_by_class_name('_doc79r').text[1:])
    except: 
        pass
    
    try:
        guests = int(driver.find_elements_by_class_name('_czm8crp')[1].text.split()[0])
    except: 
        pass
    
    try:
        bedrooms = str(driver.find_elements_by_class_name('_czm8crp')[2].text.split()[0])
    except:
        pass
    
    try:
        beds = int(driver.find_elements_by_class_name('_czm8crp')[3].text.split()[0])
    except:
        pass    
    
    try:
        baths = float(driver.find_elements_by_class_name('_czm8crp')[4].text.split()[0])
    except:
        pass   
    
    try:
        property_type = str(driver.find_elements_by_class_name('_1p3joamp')[0].text)
    except: 
        pass    
    
    try:
        location = driver.find_element_by_xpath('//*[@id="neighborhood"]/div/div/div/div/div/section/div[2]/div[2]/div[1]').get_attribute('innerHTML')
        
        loc_str = re.search('(?<=maps/@).*(?=/data=)', location)[0]

        latitude = float(loc_str.split(',')[0])
        longitude = float(loc_str.split(',')[1])
    except: 
        pass   
    
    try:
        review_rate = float(driver.find_element_by_class_name('_tghtxy2').text)
    except:
#     except erorrs as er: 
        pass 

    
    try:
        to_num = lambda num: int(''.join(i for i in num)) 
        review_num = to_num(re.findall(r'\d', driver.find_element_by_class_name('_1plk0jz1').text))
#         review_num = to_num(re.findall(r'\d', driver.find_elements_by_class_name('_krjbj')[4].text))
    except:
        pass
    
# add data in a row fashion to a dataframe
    row = {
        'property_id': property_id,
        'price': price, 
        'guests':guests, 
        'bedrooms':bedrooms, 
        'beds':beds, 
        'baths':baths,  
        'latitude': latitude,
        'longitude': longitude,
        'review_rate':review_rate, 
        'review_num':review_num, 
        'property_type':property_type,
        'link': link
    }
    driver.close()

    return row

In [559]:
cols = ['property_id', 'price', 'guests', 'bedrooms', 'beds', 
        'baths', 'latitude', 'longitude', 'review_rate', 'review_num', 
        'property_type', 'link']

df_raw = pd.DataFrame(columns = cols)

for prop_id, url in dict_combined.items():
    row = get_raw_feature_row(prop_id, url)
    df_raw = df_raw.append(row, ignore_index=True)

In [564]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 12 columns):
property_id      1636 non-null object
price            1519 non-null float64
guests           1506 non-null float64
bedrooms         1634 non-null object
beds             1525 non-null float64
baths            1514 non-null float64
latitude         1525 non-null float64
longitude        1525 non-null float64
review_rate      1491 non-null float64
review_num       1383 non-null float64
property_type    1634 non-null object
link             1636 non-null object
dtypes: float64(8), object(4)
memory usage: 153.5+ KB


In [565]:
# pickle df_raw
with open('df_raw.pkl', 'wb') as f:
    pickle.dump(df_raw, f)

In [574]:
df_raw.info()
df_raw['property_id'].value_counts()
len(df_raw['property_id'].unique())
df_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 12 columns):
property_id      1636 non-null object
price            1519 non-null float64
guests           1506 non-null float64
bedrooms         1634 non-null object
beds             1525 non-null float64
baths            1514 non-null float64
latitude         1525 non-null float64
longitude        1525 non-null float64
review_rate      1491 non-null float64
review_num       1383 non-null float64
property_type    1634 non-null object
link             1636 non-null object
dtypes: float64(8), object(4)
memory usage: 153.5+ KB


property_type    Entire home
Name: 1, dtype: object

In [7]:
# collect raw data for seattle location
for url in seattle_prop_urls:
    row = get_raw_feature_row(url)
    df_seattle = df_seattle.append(row, ignore_index=True)

In [10]:
# pickle df_raw
with open('df_seattle.pkl', 'wb') as f:
    pickle.dump(df_seattle, f)

In [45]:
# collect raw data for shoreline location
cols = ['link', 'price', 'guests', 'bedrooms', 'beds', 'baths', 
        'review_rate', 'review_num', 'property_type']
df_shoreline = pd.DataFrame(columns = cols)

# collect raw data
for url in shoreline_prop_urls_un:
    row = get_raw_feature_row(url)
    df_shoreline = df_shoreline.append(row, ignore_index=True)

In [ ]:
# pickle df_bellevue
with open('df_shoreline.pkl', 'wb') as f:
    pickle.dump(df_shoreline, f)

In [61]:
# collect raw data for bellevue location
cols = ['link', 'price', 'guests', 'bedrooms', 'beds', 'baths', 
        'review_rate', 'review_num', 'property_type']
df_bellevue = pd.DataFrame(columns = cols)

# collect raw data
for url in bellevue_prop_urls_un:
    row = get_raw_feature_row(url)
    df_bellevue = df_bellevue.append(row, ignore_index=True)

In [64]:
# pickle df_bellevue
with open('df_bellevue.pkl', 'wb') as f:
    pickle.dump(df_bellevue, f)

In [70]:
# collect raw data for bellevue location
cols = ['link', 'price', 'guests', 'bedrooms', 'beds', 'baths', 
        'review_rate', 'review_num', 'property_type']
df_redmond = pd.DataFrame(columns = cols)

# collect raw data
for url in redmond_prop_urls_un:
    row = get_raw_feature_row(url)
    df_redmond = df_redmond.append(row, ignore_index=True)

In [72]:
# pickle df_redmond
with open('df_redmond.pkl', 'wb') as f:
    pickle.dump(df_redmond, f)

In [90]:
# collect raw data for bellevue location
cols = ['link', 'price', 'guests', 'bedrooms', 'beds', 'baths', 
        'review_rate', 'review_num', 'property_type']
df_seattle2 = pd.DataFrame(columns = cols)

# collect raw data
for url in redmond_prop_urls_un:
    row = get_raw_feature_row(url)
    df_seattle2 = df_seattle2.append(row, ignore_index=True)

In [107]:
# pickle df_redmond
with open('df_seattle2.pkl', 'wb') as f:
    pickle.dump(df_seattle2, f)

In [109]:
df_raw = pd.concat([df_seattle, df_seattle2, df_shoreline, df_bellevue, df_redmond])

In [112]:
df_raw.dropna().shape

(868, 9)

In [103]:
{x:type(x) for x in df_seattle2.price.unique() if not type(x) == int}

{nan: float}

In [ ]:
url = list(seattle_prop_urls)[37]
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2) 

In [ ]:
url = list(seattle_prop_urls)[36]
driver = webdriver.Chrome()
driver.get(url)
time.sleep(30) 

# coordinates = driver.find_elements_by_class_name('_e296pg')



src = driver.find_element_by_xpath('//*[@id="neighborhood"]/div/div/div/div/div/section/div[2]/div[2]/div[2]/img').get_attribute("src")
src.location_once_scrolled_into_view



In [ ]:
# collect raw data
for url in seattle_prop_urls:
    row = get_raw_feature_row(url)
    df_raw = df_raw.append(row, ignore_index=True)

In [ ]:
# pickle df_raw
with open('df_raw.pkl', 'wb') as f:
    pickle.dump(df_raw, f)

In [ ]:
df_raw.shape